## Submission Notebook Template 

<h3> <font color='red'> WARNING : </font>  </h3>

<font color='red'> No matter which approach you've chosen, you need to re-install any custom packages you had to install to make your code work ! </font>

<b> Install your packages below: </b>

In [ ]:
!pip install git+https://github.com/Total-RD/pymgrid/
## Other packages 

In the section below, you must run your methodology for solving the problem from start to finish :

In [ ]:
import pickle

with open('building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)

with open('building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    
with open('building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)

buildings = [building_1, building_2, building_3]

<h2> Evaluation for Rule Based Approaches </h2>

If you're using Rule Based Algorithms, here is what your submitted code should feature no matter what 

<b> 1) Import all used libraries and scripts here </b>

In [ ]:
import time # Necessary to evaluate frugality
import json # Necessary to export your results
## Other packages 

<b> 2) Implementation of the rules that generate control dictionaries </b>

In [ ]:
"""
EXAMPLE FOR A NAIVE RULE BASED STRATEGY
"""
def naive_rule_based_strategy(building):
    building_data = building.get_updated_values()

    total_building_cost = 0

    while building.done == False:
        load = building_data['load']
        pv = building_data['pv']
        capa_to_charge = building_data['capa_to_charge']
        capa_to_dischare = building_data['capa_to_discharge']

        p_disc = max(0, min(load-pv, capa_to_dischare, building.battery.p_discharge_max))
        p_char = max(0, min(pv-load, capa_to_charge, building.battery.p_charge_max))

        if load - pv >=  0:

            control_dict = {'battery_charge': 0,
                                'battery_discharge': p_disc,
                                'grid_import': max(0, load-pv-p_disc),
                                'grid_export':0,
                                'pv_consummed': min(pv, load),
                           }


        if load - pv <  0:

            control_dict = {'battery_charge': p_char,
                                'battery_discharge': 0,
                                'grid_import': 0,
                                'grid_export': max(0,pv-load-p_char),#abs(min(load-pv,0)),
                                'pv_consummed': min(pv, load+p_char),
                           }

        building_data = building.run(control_dict)
        total_building_cost += building.get_cost()
    
    return total_building_cost

<b> 3) Run of the rules on the Test environment </b>

In [ ]:
"""
Rule based methods have no "training" as such, this means Training CPU Time will always be 0 and only
Test CPU Time will represent frugality
"""

eval_start = time.process_time()

total_building_costs = []

for building in buildings:

    building.reset(testing = True)

    total_building_cost = naive_rule_based_strategy(building)
    total_building_costs.append(total_building_cost)

eval_end = time.process_time()



In [ ]:
total_cost_building_1 = total_building_costs[0]
total_cost_building_2 = total_building_costs[1]
total_cost_building_3 = total_building_costs[2]

In [ ]:
frugality = eval_end - eval_start

<b> 4) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost_building_1,
    "building_2_performance" : total_cost_building_2,
    "building_3_performance" : total_cost_building_3,
    "frugality" : frugality,
}
print(final_results)

In [ ]:
team_name = ## Enter Team name here

with open(team_name + '.txt', 'w') as json_file:
    json.dump(final_results, json_file)

<h2> Evaluation for "Simple" Reinforcement Learning based approaches <h2>

 <font color='red'> <b> Be careful, the rewards returned by the Gym environment are negative ! Don't forget to multiply by -1 the total reward as the Profitability you will need to submit needs to be positive ! </b> </font>

<b> 1) Import all used libraries and scripts here </b>

In [ ]:
import time # Necessary to evaluate frugality
import json # Necessary to export your results
import DiscreteEnvironment as DiscreteEnvironment # Imposed Discrete Environment
import numpy as np

## Other packages

<b> 2) Agent & Environment Setup before your training </b>

In [ ]:
building_environments = [
    DiscreteEnvironment.Environment(env_config={'building':buildings[i]}) for i in range(3)
]
"""
Agent, potential Q-Table & other necessary setup code here 
"""


<b> 3) Training of the agent </b>

In [ ]:

train_start = time.process_time()

"""
Training code
"""

train_end = time.process_time()



In [ ]:
train_frugality = train_end - train_start

<b> 4) Test of the agent </b>

In [ ]:
"""
Below is an example for a Random Agent 

Note :
* To make your work as reproductible as possible, have a full-greedy approach (no exploration) on the test buildings
* If your algorithm has some unavoidable randomness, consider running it for many loops and return a
  mean profitability and mean frugality
  
"""

test_start = time.process_time()
total_cost = [0,0,0]

for i,building_env in enumerate(building_environments):
    
    obs = building_env.reset(testing=True)
    done = False
    while not done:
        action = np.random.randint(building_env.action_space.n)
        obs, reward, done, info = building_env.step(action)
        total_cost[i]+=reward

test_end = time.process_time()

In [ ]:
test_frugality = test_end - test_start


<b> 5) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost[0],
    "building_2_performance" : total_cost[1],
    "building_3_performance" : total_cost[2],
    "frugality" : train_frugality + test_frugality,
}
print(final_results)

In [ ]:
team_name = ## Enter Team name here

with open(team_name + '.txt', 'w') as json_file:
    json.dump(final_results, json_file)

<h2> Evaluation for Deep Reinforcement Learning based approaches <h2>

 <font color='red'> <b> Be careful, the rewards returned by the Gym environment are negative ! Don't forget to multiply by -1 the total reward as the Profitability you will need to submit needs to be positive ! </b> </font>

<b> 1) Import all used libraries and scripts here </b>

In [ ]:
import time # Necessary to evaluate frugality
from pymgrid.Environments.pymgrid_cspla import MicroGridEnv # Imposed Environment
import numpy as np

## Import your favourite Deep Learning library for RL and other packages here

<b> 2) Agent & Environment Setup before your training </b>

In [ ]:
"""
Below is an environment initialization without a Deep RL library, the code can vary depending on which library you 
use
"""
building_environments = [MicroGridEnv(env_config={'microgrid':buildings[i]}) for i in range(3)]


<b> 3) Training of the agent </b>

In [ ]:
train_start = time.process_time()

"""
Train code
"""

train_end = time.process_time()
train_frugality = train_end - train_start


frugality = train_frugality + test_frugality

<b> 4) Test of the agent </b>

In [ ]:
"""
Below is an example for a Random Agent 

Note :
* To make your work as reproductible as possible, have a full-greedy approach (no exploration) on the test buildings
* If your algorithm has some unavoidable randomness, consider running it for many loops and return a
  mean profitability and mean frugality
  
"""

test_start = time.process_time()


test_start = time.process_time()
total_cost = [0,0,0]

for i,building_env in enumerate(building_environments):
    
    obs = building_env.reset(testing=True)
    done = False
    while not done:
        action = np.random.randint(building_env.action_space.n)
        obs, reward, done, info = building_env.step(action)
        total_cost[i]+=reward

test_end = time.process_time()

test_frugality = test_end - test_start

<b> 5) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost[0],
    "building_2_performance" : total_cost[1],
    "building_3_performance" : total_cost[2],
    "frugality" : train_frugality + test_frugality,
}
print(final_results)

In [ ]:
team_name = ## Enter Team name here

with open(team_name + '.txt', 'w') as json_file:
    json.dump(final_results, json_file)

<h2> For Optimisation based approaches <h2>

As explained in the README and the Getting Started Notebook, the evaluation depends heavily on how you formulate the problem, no template can be given for this familly of methods 